In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy.stats import pearsonr
%matplotlib inline 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [2]:
df=pd.read_csv('car_ad.csv',encoding='cp1252')

In [3]:
# df['pricerange']= pd.Series(np.random.randn(df.shape[0]), index=df.index)
df.head(10)

,car,price,body,mileage,engV,engType,registration,year,model,drive
0,Ford,15500.0,crossover,68,2.5,Gas,yes,2010,Kuga,full
1,Mercedes-Benz,20500.0,sedan,173,1.8,Gas,yes,2011,E-Class,rear
2,Mercedes-Benz,35000.0,other,135,5.5,Petrol,yes,2008,CL 550,rear
3,Mercedes-Benz,17800.0,van,162,1.8,Diesel,yes,2012,B 180,front
4,Mercedes-Benz,33000.0,vagon,91,NaN,Other,yes,2013,E-Class,NaN
5,Nissan,16600.0,crossover,83,2.0,Petrol,yes,2013,X-Trail,full
6,Honda,6500.0,sedan,199,2.0,Petrol,yes,2003,Accord,front
7,Renault,10500.0,vagon,185,1.5,Diesel,yes,2011,Megane,front
8,Mercedes-Benz,21500.0,sedan,146,1.8,Gas,yes,2012,E-Class,rear
9,Mercedes-Benz,22700.0,sedan,125,2.2,Diesel,yes,2010,E-Class,rear


In [5]:
try:
    with open('car_ad.csv', 'r') as f:
        ...
except OSError:
    # 'File not found' error message.
    print("File not found")

In [6]:
print(f)

<_io.TextIOWrapper name='car_ad.csv' mode='r' encoding='cp1252'>


In [7]:
# df.sort_values('price',ascending=True)[250:500]#.head(500)
np.amin(df.price)

0.0

In [8]:
new_df = df[['car','body','model']]
new_df.head()

,car,body,model
0,Ford,crossover,Kuga
1,Mercedes-Benz,sedan,E-Class
2,Mercedes-Benz,other,CL 550
3,Mercedes-Benz,van,B 180
4,Mercedes-Benz,vagon,E-Class


In [9]:
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
#new_df.dropna(inplace=True)

blanks = []  # start with an empty list

col=['car','body','model']
for i,col in new_df.iterrows(): 
    if type(col)==str:          
        if col.isspace():         
            blanks.append(i)    

#new_df.drop(blanks, inplace=True)

In [11]:
new_df.set_index('car', inplace = True)
new_df.head()

,body,model
car,,
Ford,crossover,Kuga
Mercedes-Benz,sedan,E-Class
Mercedes-Benz,other,CL 550
Mercedes-Benz,van,B 180
Mercedes-Benz,vagon,E-Class


In [12]:
new_df['bag_of_words'] = ''
columns = new_df.columns
for index, row in new_df.iterrows():
    words = ''
    for col in columns:
        if col != 'model':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
new_df.drop(columns = [col for col in new_df.columns if col!= 'bag_of_words'], inplace = True)

C:\Users\KIIT\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\KIIT\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [13]:
new_df.head()

,bag_of_words
car,
Ford,c r o s s o v e r Kuga
Mercedes-Benz,s e d a n E-Class
Mercedes-Benz,o t h e r CL 550
Mercedes-Benz,v a n B 180
Mercedes-Benz,v a g o n E-Class


In [14]:
count = CountVectorizer()
count_matrix = count.fit_transform(new_df['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(new_df.index)
indices[:5]
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 1. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 1. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 1. , 0.5, 0. ],
       [0. , 0. , 0. , ..., 0.5, 1. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 1. ]])

In [15]:
def recommendations(Title, cosine_sim = cosine_sim):
    
    recommended_cars = []
    
   
    idx = indices[indices == Title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10  similar cars
    top_10_indexes = list(score_series.iloc[1:6].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_cars.append(list(df.index)[i])
        
    return recommended_cars

def rem(Title):
    y= recommendations(Title)
    rec= []
    for i in range(len(y)):
        rec.append(df.index[y[i]])
    return rec

In [16]:
rem('UAZ')

[7852, 4733, 4344, 3186, 3192]

In [17]:
rem('Honda')

[311, 5724, 965, 2141, 2129]

In [18]:
count = CountVectorizer()
count_matrix = count.fit_transform(new_df['bag_of_words'])

In [19]:
count_matrix.toarray()[0].shape

(752,)

In [20]:
np.unique(df.model).shape

(888,)

In [44]:
from tqdm import tqdm
new_df= pd.read_csv('car_ad.csv',encoding='cp1252')
new_df['new_col'] = range(1, len(new_df) + 1)
new_df.set_index('new_col', inplace=True)
new_df.head()


,car,price,body,mileage,engV,engType,registration,year,model,drive
new_col,,,,,,,,,,
1,Ford,15500.0,crossover,68,2.5,Gas,yes,2010,Kuga,full
2,Mercedes-Benz,20500.0,sedan,173,1.8,Gas,yes,2011,E-Class,rear
3,Mercedes-Benz,35000.0,other,135,5.5,Petrol,yes,2008,CL 550,rear
4,Mercedes-Benz,17800.0,van,162,1.8,Diesel,yes,2012,B 180,front
5,Mercedes-Benz,33000.0,vagon,91,NaN,Other,yes,2013,E-Class,NaN


In [25]:
new_df= new_df.iloc[:9576,:-5]
i= 0
r= []
for _ in tqdm(range(0, len(new_df.index))):
    temp= rem(new_df.car.tolist()[i])
    r.append(temp);
    i+= 1  

100%|██████████████████████████████████████████████████████████████████████████████| 9576/9576 [02:36<00:00, 61.38it/s]


In [43]:
r.index+=1
r

,0,1,2,3,4
1,5040,1073,6603,2588,7298
2,4182,2279,211,834,1722
3,4182,2279,211,834,1722
4,4182,2279,211,834,1722
5,4182,2279,211,834,1722
6,668,1257,21,229,9495
7,311,5724,965,2141,2129
8,1498,6114,6774,961,6364
9,4182,2279,211,834,1722
10,4182,2279,211,834,1722


In [45]:
new_df= pd.concat([new_df, r], axis=1, ignore_index=True)

In [47]:
new_df.to_csv("Car_recommendation.csv")

In [48]:
new_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
1,Ford,15500.000,crossover,68,2.50,Gas,yes,2010,Kuga,full,5040,1073,6603,2588,7298
2,Mercedes-Benz,20500.000,sedan,173,1.80,Gas,yes,2011,E-Class,rear,4182,2279,211,834,1722
3,Mercedes-Benz,35000.000,other,135,5.50,Petrol,yes,2008,CL 550,rear,4182,2279,211,834,1722
4,Mercedes-Benz,17800.000,van,162,1.80,Diesel,yes,2012,B 180,front,4182,2279,211,834,1722
5,Mercedes-Benz,33000.000,vagon,91,NaN,Other,yes,2013,E-Class,NaN,4182,2279,211,834,1722
6,Nissan,16600.000,crossover,83,2.00,Petrol,yes,2013,X-Trail,full,668,1257,21,229,9495
7,Honda,6500.000,sedan,199,2.00,Petrol,yes,2003,Accord,front,311,5724,965,2141,2129
8,Renault,10500.000,vagon,185,1.50,Diesel,yes,2011,Megane,front,1498,6114,6774,961,6364
9,Mercedes-Benz,21500.000,sedan,146,1.80,Gas,yes,2012,E-Class,rear,4182,2279,211,834,1722
10,Mercedes-Benz,22700.000,sedan,125,2.20,Diesel,yes,2010,E-Class,rear,4182,2279,211,834,1722


In [50]:
new_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
1,Ford,15500.0,crossover,68,2.5,Gas,yes,2010,Kuga,full,5040,1073,6603,2588,7298
2,Mercedes-Benz,20500.0,sedan,173,1.8,Gas,yes,2011,E-Class,rear,4182,2279,211,834,1722
3,Mercedes-Benz,35000.0,other,135,5.5,Petrol,yes,2008,CL 550,rear,4182,2279,211,834,1722
4,Mercedes-Benz,17800.0,van,162,1.8,Diesel,yes,2012,B 180,front,4182,2279,211,834,1722
5,Mercedes-Benz,33000.0,vagon,91,NaN,Other,yes,2013,E-Class,NaN,4182,2279,211,834,1722


In [52]:
new_df=pd.read_csv('Car_recommendation.csv')

In [53]:
new_df.head()

,Sl No.,car,price,body,mileage,engV,engType,registration,year,model,drive,R1,R2,R3,R4,R5
0,1,Ford,15500.0,crossover,68,2.5,Gas,yes,2010,Kuga,full,5040,1073,6603,2588,7298
1,2,Mercedes-Benz,20500.0,sedan,173,1.8,Gas,yes,2011,E-Class,rear,4182,2279,211,834,1722
2,3,Mercedes-Benz,35000.0,other,135,5.5,Petrol,yes,2008,CL 550,rear,4182,2279,211,834,1722
3,4,Mercedes-Benz,17800.0,van,162,1.8,Diesel,yes,2012,B 180,front,4182,2279,211,834,1722
4,5,Mercedes-Benz,33000.0,vagon,91,NaN,Other,yes,2013,E-Class,NaN,4182,2279,211,834,1722
